# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [124]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [125]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))


In [126]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get transfers and trades

In [127]:
df_trade = pd.read_sql_query('SELECT * FROM trades', con)
df_transfer = pd.read_sql_query('SELECT * FROM transfers', con)


In [128]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [131]:
df_trade_summ.sort_values('feesPaid', ascending=False)

,asset,feesPaid
22,sETH,983702.189135
21,sBTC,866069.437568
16,OP,214450.434311
17,SOL,119272.348969
13,LINK,98062.744667
14,MATIC,92247.969698
6,BNB,57808.827489
11,FTM,53974.320854
4,AVAX,44382.331546
8,DYDX,36827.592713


## Get the market data

In [132]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [133]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
df_markets

,asset,marketDebt
0,sETH,6.801638e+06
1,sBTC,5.997886e+06
2,LINK,2.432271e+05
3,SOL,4.672828e+05
4,AVAX,3.425052e+05
5,AAVE,2.979049e+03
6,UNI,9.680554e+04
7,MATIC,4.792965e+05
8,APE,5.564707e+04
9,DYDX,2.527864e+04


In [134]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,6.801638e+06,7.550326e+06,983702.189135,-1.732390e+06
1,sBTC,5.997886e+06,6.339045e+06,866069.437568,-1.207229e+06
2,LINK,2.432271e+05,3.396018e+05,98062.744667,-1.944374e+05
3,SOL,4.672828e+05,5.409901e+05,119272.348969,-1.929797e+05
4,AVAX,3.425052e+05,2.694674e+05,44382.331546,2.865549e+04
5,AAVE,2.979049e+03,1.172940e+03,5111.573890,-3.305465e+03
6,UNI,9.680554e+04,1.101012e+05,10963.379048,-2.425904e+04
7,MATIC,4.792965e+05,5.862391e+05,92247.969698,-1.991905e+05
8,APE,5.564707e+04,2.557363e+04,31144.923675,-1.071482e+03
9,DYDX,2.527864e+04,5.046662e+04,36827.592713,-6.201557e+04


In [135]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [136]:
df_pnl['netPnl'].sum()

-4208876.324164831

In [137]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-1732389.95
1,sBTC,-1207229.10
11,OP,-319263.87
7,MATIC,-199190.51
2,LINK,-194437.44
3,SOL,-192979.66
19,FTM,-142913.57
10,BNB,-73449.99
9,DYDX,-62015.57
16,ATOM,-48950.29
